## Purpose of this Notebook

This notebook demonstrates how to extract text content from PDF files and store it into a MySQL database.  
It covers the workflow of reading PDF documents, processing the extracted text, and inserting structured information into the database, enabling further analysis and retrieval of document contents.


In [ ]:
import mysql.connector
import pdfplumber
import io
import dbtools
import asyncio
from pypdf import PdfReader
from tika import parser

In [ ]:
def connect_to_database():
    # Replace with your database connection details
    return mysql.connector.connect(
        host="YOUR_HOST",
        user="YOUR_USERNAME",
        password="YOUR_PASSWORD",
        database="YOUR_DATABASE",
        port=3306
    )

def pdf_extractor(row_id):

    conn = conn = connect_to_database()
    cursor = conn.cursor()

    
    query = f"SELECT prof_id,raw_file,id FROM raw_files WHERE id = {row_id} AND file_content = 'cv'"

    cursor.execute(query)

    # Fetch the results
    rows = cursor.fetchall()

    # Check if the result is an empty list
    if not rows:
        s = "No rows were found with the specified conditions."
        # Close the cursor and connection
        cursor.close()
        conn.close()
        print(s)
        return 
    
    else:
        
        text = ""
        
        for row in rows:

            try:
            
                pdf_data = row[1]
                pdf_stream = io.BytesIO(pdf_data)

                raw = parser.from_buffer(pdf_stream)
                text = text + raw['content'].strip() + "\n"

            
            except:

                print(f"error in parsing prof_id:{row[0]} and document_id:{row[2]}")
                
        # Close the cursor and connection
        cursor.close()
        conn.close()
        return text

In [ ]:
conn = connect_to_database()
cursor = conn.cursor()

query = f"SELECT id FROM raw_files WHERE file_content = 'cv'"

cursor.execute(query)

# Fetch the results
rows = cursor.fetchall()

ids = [row[0] for row in rows]

ids = sorted(list(set(ids)))
cursor.close()
conn.close()

In [ ]:
def update(row_id,text):
    conn = connect_to_database()
    cursor = conn.cursor()
    table_name = "individual_crawl.raw_files"
    column_name = "text" 
    row_id = row_id
    query = f"UPDATE {table_name} SET {column_name} = %s WHERE id = %s"

  

    try:
        # Execute the query
        cursor.execute(query, (text, row_id))
        # Commit the changes
        conn.commit()

    except mysql.connector.Error as err:
        print(f"\nid: {row_id} Error: {err}")
        conn.rollback()

    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

In [ ]:
n = len(ids)
for index,row_id in enumerate(ids):
    
    t = pdf_extractor(row_id)

    update(row_id,t)

    print(f"\r{index}/{n}" , end="")

